In [2]:
import os 
import sys
import subprocess
import pandas as pd
import numpy as np
import pybedtools
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from gene2probe import *

/nfs/team205/is10/miniconda/envs/gene2probe/lib/python3.11/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [12]:
gtf = '/nfs/team205/is10/resources/ucsc_genomes/hg38.ncbiRefSeq.gtf' ## Gene annotation in gtf file
fasta = '/nfs/team205/is10/resources/ucsc_genomes/hg38.fa' ## Genome in fasta file
mode = 'exon'
out_dir = '/nfs/team205/is10/projects/thytrans/data/gene2probe/blastdb/'
os.makedirs(out_dir , exist_ok=True)
out_name = 'hg38_ncbiRefSeq_exons'
blast_exec_path = '/nfs/team205/is10/miniconda/envs/gene2probe/bin/'

In [4]:
## Read gtf
gene_anno = read_gtf(gtf)

## Filter for exon/transcript respectively
gene_anno = gene_anno[gene_anno['feature'] == mode].reset_index(drop=True)

## Extract gene name
gene_ids = gene_anno['attribute'].apply(extract_feature_from_gtf, feature='gene_name')

## Convert to bed
bed_df = gtf_2_bed(gene_anno, name_pref = (''))
bed_df['name'] = gene_ids

## Initialise bedtools
bed = pybedtools.BedTool.from_dataframe(bed_df)

/nfs/team205/is10/projects/thytrans/scripts/008_visiumHD/gene2probe.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['start'] = df_bed['start'].astype(int) - 1


In [5]:
gene_anno.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,chrM,ncbiRefSeq.2022-10-28,exon,15956,16023,.,-,.,"gene_id ""TRNP""; transcript_id ""rna-TRNP""; exon..."
1,chrM,ncbiRefSeq.2022-10-28,exon,15888,15953,.,+,.,"gene_id ""TRNT""; transcript_id ""rna-TRNT""; exon..."
2,chrM,ncbiRefSeq.2022-10-28,exon,14747,15887,.,+,.,"gene_id ""CYTB""; transcript_id ""rna-CYTB""; exon..."
3,chrM,ncbiRefSeq.2022-10-28,exon,14674,14742,.,-,.,"gene_id ""TRNE""; transcript_id ""rna-TRNE""; exon..."
4,chrM,ncbiRefSeq.2022-10-28,exon,14149,14673,.,-,.,"gene_id ""ND6""; transcript_id ""rna-ND6""; exon_n..."


In [7]:
## Get fasta
seq = bed.sequence(fi=fasta, s=True, name=True, fullHeader=True) 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
##Export fasta
seq.save_seqs((out_dir + out_name +  '.fa'))

<BedTool(/tmp/pybedtools.92zqfjqx.tmp)>

In [13]:
## Make blast db
command = [
    (blast_exec_path + 'makeblastdb'),
    '-in', (out_dir + out_name +  '.fa'),  # Input FASTA file
    '-dbtype', 'nucl',                     # Database type, 'nucl' for nucleotide
    '-out', (out_dir + out_name +  '_db')  # Output database name
]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Database created successfully!")
    print(result.stdout)  # Print standard output
else:
    print("Error in database creation:")
    print(result.stderr)  # Print any error messages

Database created successfully!


Building a new DB, current time: 04/24/2024 22:10:36
New DB name:   /nfs/team205/is10/projects/thytrans/data/gene2probe/blastdb/hg38_ncbiRefSeq_exons_db
New DB title:  /nfs/team205/is10/projects/thytrans/data/gene2probe/blastdb/hg38_ncbiRefSeq_exons.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2178727 sequences in 75.0933 seconds.

